Poisson Regression

Purpose: Count data, assuming the variable Y has a Poisson Distribution

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.special import factorial

In [18]:
#data train
file_train='D:\\Project\\Data_Mining\\Finals_BlogCommentPrediction\\Train\\blogData_train.csv'
train=pd.read_csv(file_train, header=None)

#data test
file_test='D:\\Project\\Data_Mining\\Finals_BlogCommentPrediction\\Test\\blogData_test.csv'
test=pd.read_csv(file_test, header=None)

train = train.drop(train.columns[:50], axis=1)  # Drop the first 50 columns
test = test.drop(test.columns[:50], axis=1)  # Drop the first 50 columns

# Split features and labels
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

C:\Users\scarl\AppData\Local\Temp\ipykernel_2512\3327860895.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test=pd.read_csv(file_test, header=None)


In [19]:
class PoissonRegression:
    def __init__(self, learning_rate=0.01, max_iter=10000, tol=1e-7):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.beta = None

    def _log_likelihood(self, beta, X, y):
        linear_pred = np.dot(X, beta)
        lambda_ = np.exp(linear_pred)
        logL = np.sum(y * linear_pred - lambda_ - np.log(factorial(y)))
        return -logL  # We return the negative log-likelihood for minimization

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.beta = np.zeros(n_features)
        
        # Using minimize function from scipy to find the optimal parameters
        result = minimize(self._log_likelihood, self.beta, args=(X, y), method='BFGS', options={'disp': False})
        self.beta = result.x
        
        return self
    
    def loss_function(self, w, X, y):
        y_pred = np.exp(np.dot(X, w))
        loss = ((y_pred - y) ** 2.0)
        return loss.mean(axis=None)

    def predict(self, X):
        # Fill NaN values in X with the mean of each column
        col_means = np.nanmean(X, axis=0)
        X = np.where(np.isnan(X), col_means, X)
        
        linear_pred = np.dot(X, self.beta)
        return np.exp(linear_pred)

In [20]:
# Print the sum of NaN values to confirm their presence
print("NaN values in y_test before:", y_test.isna().sum())

# Replace NaN with 1
y_test.fillna(1, inplace=True)

# Verify the replacement
print("NaN values in y_test after:", y_test.isna().sum())

print(y_test)

NaN values in y_test before: 1
NaN values in y_test after: 0
0       4.0
1       0.0
2       1.0
3       5.0
4       0.0
       ... 
7620    0.0
7621    0.0
7622    3.0
7623    0.0
7624    1.0
Name: 280, Length: 7625, dtype: float64


In [25]:
model = PoissonRegression(max_iter=10, tol=0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

C:\Users\scarl\AppData\Roaming\Python\Python312\site-packages\scipy\optimize\_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


array([1., 1., 1., ..., 1., 1., 1.])

In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Root Mean Squared Error


print("Mean Squared Error:", mse)
print(f'RMSE : {rmse}')

Mean Squared Error: 948.9826885245901
RMSE : 30.805562623081407


In [27]:
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')

Mean Absolute Error (MAE): 5.746229508196722


In [24]:
import joblib

joblib.dump(model, 'poisson_regression.joblib')

['poisson_regression.joblib']